# Statistical analysis in Python using Pingouin

In [ ]:
import numpy as np
import pandas as pd
import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns
from statannot import add_stat_annotation

import biopsykit as bp

from IPython.display import Markdown

%load_ext autoreload
%autoreload 2
%matplotlib widget

**Pingouin**: Open source library for statistical analyses in Python, based on *pandas* and *numpy*.

Installation: `pip install pingouin`  
Link: https://pingouin-stats.org/  
Tutorial: https://pingouin-stats.org/guidelines.html

Import: `import pingouin as pg`

In [ ]:
sns.set_theme(style="ticks")
sns.set_palette(sns.color_palette("light:#003865", n_colors=4)[::-1])
plt.close('all')

# parameter for add_stat_annotation
pvalue_thres = [[1e-3, "***"], [1e-2, "**"], [0.05, "*"]]

<div class="alert alert-block alert-info">
    <b>Note:<b> Just like Seaborn, it is also advantageous for Pingouin if the data is in <i>long format<i>!
</div>

## Importing data

Dataset from Richer et al. "Tackling Acute Stress: Evaluating the Cold Face Test as a Potential Mechanism for Stress-reducing Wearables"
HR(V), cortisol and questionnaires during the implementation of MIST with two groups: Control group and intervention group
* HR(V): Heart rate (HR), RMSSD, pNN50, SD1/SD2 variables per MIST phase (1-3) and per MIST subphase (baseline (BL), cold face intervention (CFI), arithmetic tasks (AT), feedback (FB)), each absolute and relative (e.g., $\text{HR}_{rel}$, normalized per subject before beginning of the stressor)
* Questionnaire: MDBF (Multidimensional Sensitivity Questionnaire, both *pre* and *post* stress)
* Cortisol: 7 saliva samples at time points -27 (S0, to control for high baseline), 0, 30, 40, 50, 60, 70 (min relative to MIST start)

### HR(V)

In [ ]:
df_hr = pd.read_csv("data/hr_sample.csv", index_col=["condition", "subject", "phase", "subphase"])
df_hr.head()

### Questionnaires

In [ ]:
df_mood = pd.read_csv("data/mood_sample.csv", index_col=["condition", "subject", "time", "type"])
df_mood.head()

### Cortisol

In [ ]:
df_cort_raw = pd.read_csv("data/cortisol_sample.csv", index_col=["condition", "subject"])
saliva_times = [-27, 0, 30, 40, 50, 60, 70]

# Convert to long format
df_cort_raw = pd.DataFrame(df_cort_raw.stack())
df_cort_raw.columns = ["cortisol"]
df_cort_raw.index = df_cort_raw.index.rename('sample', level=2)

# Splitting of cortisol values into value before (sample 1) and maximum value per subject after (samples 2 to 6)
cort_pre = df_cort_raw.unstack(level='sample').iloc[:, 1]
cort_post = df_cort_raw.unstack(level='sample').iloc[:, 2:].max(axis=1)
df_cort_inc = pd.concat([cort_pre, cort_post], axis=1, keys=['pre', 'post'], names=['time'])
df_cort_inc = pd.DataFrame(df_cort_inc.stack(), columns=["cortisol"])

# Calculating cortisol features
df_cort = bp.saliva.auc(df_cort_raw, saliva_times=saliva_times)
df_cort = df_cort.join(bp.saliva.max_increase(df_cort_raw, percent=True))
df_cort = df_cort.join(bp.saliva.slope(df_cort_raw, sample_idx=[1, 4], saliva_times=saliva_times))

# Convert feature dataframe to long format
df_cort = bp.saliva.wide_to_long(df_cort)
df_cort.head()

## Reaction to the MIST

### Question I: Does each phase of the MIST produce a significant cardiovascular stress response?

#### Examination

In the *control group*, each MIST phase is examined separately to determine whether the heart rate increases significantly between the *BL* and *AT* subphases.

#### Statistics

##### Analysis
* Preparatory analysis:
    * Test for normal distribution (Shapiro-Wilk test).
* *Test*: t-tests for dependent samples
* *DV*: HR
* *Inner subject factor*: Subphase
* *Measure of effect size*: Hedge's g

=> Analogous analysis for the other HRV parameters (significant reduction of HRV during stress?)

In [ ]:
df_hr.head()

In [ ]:
dv = "HR"
subphases = ["BL", "AT"]
# Selecting the data to be examined: Control group, all subjects, all MIST phases, subphases BL and AT
df_analysis = df_hr[[dv]].loc[pd.IndexSlice["Control", :, :, subphases]]

df_analysis.reset_index()

In [ ]:
# Test for normal distribution
df_norm = df_analysis.reset_index().groupby(by=["phase"]).apply(lambda df: pg.normality(data=df, dv=dv, group="subphase"))

# paired t-tests per MIST phase
tt_phase = df_analysis.reset_index().groupby(by=["phase"]).apply(lambda df: pg.pairwise_ttests(data=df, dv=dv, within="subphase", subject="subject", effsize="hedges"))

In [ ]:
# Output
display(Markdown("All normally distributed: **{}**".format(df_norm['normal'].all())))

display(Markdown("**paired t-tests**"))
display(tt_phase.round(3))

##### Interpretation

"Pairwise t-tests for each MIST phase individually revealed significant HR increases between *BL* and *AT* subphases indicating an effective sympathetic activation due to the stressor. Additionally, significant decreases of HRV measures occured in some, but not all, *MIST phases* (Table 2)."

#### Boxplot

In [ ]:
fig, ax = plt.subplots()
sns.boxplot(data=df_analysis.reset_index(), x="phase", y=dv, hue="subphase")
ax.set_ylabel("HR [bpm]")

fig.tight_layout()

**Adding significance brackets**

By means of the function `add_stat_annotation()` of the library `statannot` "significance brackets" can be added to the boxplots.

Installation: `pip install statannot`  
Documentation: https://github.com/webermarcolivier/statannot  
Examples: https://github.com/webermarcolivier/statannot/blob/master/example/example.ipynb

`add_stat_annotation()` requires a list of names of boxplot pairs over which brackets are to be placed.

List format => From documentation of the function:
* For non-grouped boxplot: `[(cat1, cat2), (cat3, cat4)]`
* For boxplot grouped by hue: `[((cat1, hue1), (cat2, hue2)), ((cat3, hue3), (cat4, hue4))]`

In [ ]:
(("MIST1", "BL"), ("MIST1", "AT"))

In [ ]:
# Significant MIST phases
sig_list = ["MIST1", "MIST2", "MIST3"]
# Constructing the boxplot pairs list
sig_pairs = [tuple([(sig, sub) for sub in subphases]) for sig in sig_list]
sig_pairs

#### Boxplot with significance brackets

In [ ]:
fig, ax = plt.subplots()
sns.boxplot(data=df_analysis.reset_index(), x="phase", y=dv, hue="subphase")
ax.set_ylabel("HR [bpm]")

add_stat_annotation(
    ax=ax, data=df_analysis.reset_index(), 
    x="phase", y=dv, hue="subphase", 
    box_pairs=sig_pairs,
    pvalue_thresholds=pvalue_thres,
    test="t-test_paired",
    comparisons_correction=None
)
fig.tight_layout()

#### Violinplot

In [ ]:
fig, ax = plt.subplots()
sns.violinplot(data=df_analysis.reset_index(), x="phase", y=dv, hue="subphase", split=True)
ax.set_ylabel("HR [bpm]")

fig.tight_layout()

### Question II: Does the stress level increase over the course of the three MIST phases?

#### Examination

In the *control group*, it is examined whether the heart rate in each of the *BL*, *AT*, and *FB* phases, respectively, increases significantly over time (over the three MIST phases).

#### Statistics

##### Analysis

* Preparatory analysis:
    * Test for normal distribution (Shapiro-Wilk test)
    * Test for equality of variance (Levene test)
* * Test: ANOVA with repeated measurements (rm-ANOVA)
* *DV*: HR, separately for each of the subphases *BL*, *AT* and *FB*
* *Inner subject factor*: Phase
* *Measure of effect size*: $\eta_p^{2}$ (partial eta-squared)
* Post hoc test: t-tests for dependent samples (one-sided) with Bonferroni correction to neutralize alpha error accumulation in multiple comparisons

=> Analogous analysis for the other HRV parameters

In [ ]:
dv = "HR"
subphases = ["BL", "AT", "FB"]
# Selecting the data to be examined: Control group, all subjects, all MIST phases, subphases BL, AT and FB
df_analysis = df_hr[[dv]].loc[pd.IndexSlice["Control", :, :, subphases]]

# Test for normal distribution
df_norm = df_analysis.reset_index().groupby(by=["subphase"]).apply(lambda df: pg.normality(data=df, dv=dv, group="phase"))
# Test for homoscedasticity
df_hom = df_analysis.reset_index().groupby(by=["subphase"]).apply(lambda df: pg.homoscedasticity(data=df, dv=dv, group="phase"))

# rm-ANOVA per subphase
anova_subph = df_analysis.reset_index().groupby(by=["subphase"]).apply(lambda df: pg.rm_anova(data=df, dv=dv, within="phase", subject="subject"))
# post-hoc
posthoc_subph = df_analysis.reset_index().groupby(by=["subphase"]).apply(lambda df: pg.pairwise_ttests(data=df, dv=dv, within="phase", subject="subject", effsize="hedges", tail='one-sided', padjust='bonf'))

# Output
display(Markdown("All normally distributed: **{}**".format(df_norm['normal'].all())))
display(Markdown("All variances (approximately) equal: **{}**".format(df_hom['equal_var'].all())))

display(Markdown("**rm-ANOVA**"))
display(anova_subph.round(3))

display(Markdown("**Post-hoc tests**"))
display(posthoc_subph.round(3))

# Optional: filtering of the significant post hoc tests
#display(posthoc_subph[posthoc_subph['p-corr'] < 0.05].round(3))

##### Interpretation

Repeated-measures *ANOVA* of HR(V) measures during *MIST subphases* indicate increasing stress levels over time, shown by a significant main effect *MIST Phase* during *BL*, as well as in *FB* and *AT* (see Table 3). Post-hoc testing of HR changes showed significant differences during *BL* between *MIST1* and *MIST3* ($t(12) = −4.285, p = 0.002, g = −0.438$) and between *MIST1* and *MIST2* ($t(12) = −2.634, p = 0.030, g = −0.384$), respectively. […]

#### Plots

In [ ]:
fig, ax = plt.subplots()
sns.boxplot(data=df_analysis.reset_index(), x="subphase", y=dv, hue="phase")

sig_pairs = [
    (("AT", "MIST1"), ("AT", "MIST2")),
    (("AT", "MIST1"), ("AT", "MIST3")),
    (("BL", "MIST1"), ("BL", "MIST3")),
    (("BL", "MIST2"), ("BL", "MIST3")),
    (("FB", "MIST1"), ("FB", "MIST3")),
    (("FB", "MIST2"), ("FB", "MIST3")),
]

add_stat_annotation(
    ax=ax, data=df_analysis.reset_index(), 
    x="subphase", y=dv, hue="phase", 
    box_pairs=sig_pairs,
    pvalue_thresholds=pvalue_thres,
    test="t-test_paired",
    comparisons_correction=None
)

ax.set_ylabel("HR [bpm]")

fig.tight_layout()

### Question III: Does the MIST worsen the mood?

#### Examination

The *control group* will be examined to determine if the mood (measured by MDBF) significantly worsened in response to the MIST.

#### Statistics

##### Analysis
* Preparatory analysis:
    * Test for normal distribution (Shapiro-Wilk test)
* *Test*: Paired t-tests
* *DV*: MDBF scales
* *Inner subject factor*: Time (pre/post)
* *Measure of effect size*: Hedge's g

In [ ]:
df_mood.head()

In [ ]:
#df_analysis.unstack(level='time')

In [ ]:
dv = "MDBF"
# Selecting the data to be studied: control group
df_analysis = df_mood.xs("Control")

# Test for normal distribution
df_norm = df_analysis.reset_index().groupby(by=["type"]).apply(lambda df: pg.normality(data=df, dv=dv, group="time"))

# paired t-tests per MDBF subscale
ttests = df_analysis.reset_index().groupby(["type"]).apply(lambda df: pg.pairwise_ttests(data=df.reset_index(), within="time", dv=dv, subject="subject", effsize="hedges"))

# Calculating the worsening of the mood in percent
df_decrease = pd.DataFrame(df_analysis.unstack(level='time').groupby("type").apply(lambda x: (x[("MDBF", 'post')] / x[("MDBF", 'pre')] - 1.0) * 100.0), columns=["decrease_percent"])
df_decrease = df_decrease.groupby("type").agg(['mean', 'std'])

# Output
display(Markdown("All normally distributed: **{}**".format(df_norm['normal'].all())))
display(df_norm)

display(Markdown("**Paired t-tests**"))
display(ttests.round(3))

display(Markdown("**Worsening of the mood (Mean ± SD)**"))
display(df_decrease.T.round(2))

##### Interpretation

"The stress-inducing effects of the *MIST* in the *Control* group were further confirmed by the MDBF questionnaire. Mood worsened significantly in all MDBF dimensions as indicated by pairwise t-tests (*Good-Bad*: $t(12) = 5.761, p < 0.001, g = 2.105$, decrease: $36.93\%±22.26\%~(\text{M}±\text{SD})$; *Awake-Tired*: $t(12) = 3.169, p = 0.008, g = 0.615$, decrease: $12.45\% ± 13.28\%$; *Calm-Nervous*: $t(12) = 5.094, p < 0.001, g = 1.855$, decrease: $35.44\% ± 23.92\%$)."

#### Plots

In [ ]:
fig, ax = plt.subplots()
x_order = ["good_bad", "awake_tired", "calm_nervous"]
sns.boxplot(data=df_analysis.reset_index(), x="type", y=dv, hue="time", order=x_order)

sig_pairs = [
    (("good_bad", "pre"), ("good_bad", "post")),
    (("awake_tired", "pre"), ("awake_tired", "post")),
    (("calm_nervous", "pre"), ("calm_nervous", "post")),
]

add_stat_annotation(
    ax=ax, data=df_analysis.reset_index(), 
    x="type", y=dv, hue="time", 
    order=x_order,
    box_pairs=sig_pairs,
    pvalue_thresholds=pvalue_thres,
    test="t-test_paired",
    comparisons_correction=None
)

fig.tight_layout()

### Question IV: Does the MIST produce a significant endocrine stress response?

#### Examination

In the *control group* it is examined, whether the cortisol level significantly increased before (S1) and after the stressor (S2-S6).

#### Statistics

##### Analysis
* Preparatory analysis:
    * Test for normal distribution (Shapiro-Wilk test)
* *Test*: Paired t-tests
* *DV*: Cortisol
* *Inner subject factor*: Time (pre/post)
* *Measure of effect size*: Hedge's g

In [ ]:
df_cort_inc.head()

In [ ]:
dv = "cortisol"
# Selecting the data to be studied: control group
df_analysis = df_cort_inc.xs("Control")

# Test for normal distribution
df_norm = pg.normality(data=df_analysis.reset_index(), dv=dv, group="time")

# paired t-tests
ttests = pg.pairwise_ttests(data=df_analysis.reset_index(), dv=dv, within="time", subject="subject", effsize="hedges")

# Calculating the cortisol increase in percent
df_increase = df_analysis.unstack(level='time')
df_increase = pd.DataFrame((df_increase[("cortisol", "post")] / df_increase[("cortisol", "pre")] - 1.0) * 100, columns=["increase_percent"])

# Output
display(Markdown("All normally distributed: **{}**".format(df_norm['normal'].all())))

display(Markdown("**paired t-tests**"))
display(ttests.round(3))

display(Markdown("**Cortisol increase**"))
display(df_increase.mean().round(2))

##### Interpretation

"Analysis of saliva samples showed that the *Control* group overall responded to the stressor. On average, cortisol levels increased significantly after the *MIST* ($t(12) = -2.284, p = 0.041, g = -0.623$) with an maximum increase of $75.4\%$ compared to S1."

## Effect of intervention on acute stress responses

### Question I: Does the MIST induce a significant cardiovascular stress response in the intervention group as well?

#### Examination

In the *intervention group* each MIST phase is examined separately to determine whether the heart rate increases significantly between the *BL* and *AT* subphases.

#### Statistics

##### Analysis

* Preparatory analysis:
    * Test for normal distribution (Shapiro-Wilk test)
* *Test*: Paired t-tests
* *DV*: HR
* *Inner subject factor*: Subphase
* *Measure of effect size*: Hedge's g (or Common Language Effect Size (CLES) for Wilcoxon test)

=> Analogous analysis for the other HRV parameters (significant reduction of HRV during stress?)

In [ ]:
dv = "HR"
subphases = ["BL", "AT"]
# Selecting the data to be examined: Control group, all subjects, all MIST phases, subphases BL and AT
df_analysis = df_hr[[dv]].loc[pd.IndexSlice["Intervention", :, :, subphases]]

# Test for normal distribution
df_norm = df_analysis.reset_index().groupby("phase").apply(
    lambda df: pg.normality(data=df, dv=dv, group="subphase")
)

# Wilcoxon tests per MIST phase
wilc_phase = df_analysis[dv].groupby("phase").apply(
    lambda df: pg.wilcoxon(
        x=df.xs("BL", level="subphase"), 
        y=df.xs("AT", level="subphase"), tail='one-sided'
    )
)

# For comparison: paired t-tests per MIST phase
tt_phase = df_analysis.reset_index().groupby("phase").apply(
    lambda df: pg.pairwise_ttests(
        data=df, dv=dv, within="subphase", 
        subject="subject", effsize="hedges", 
        tail='one-sided'
    )
)

# Output
display(Markdown("All normally distributed: **{}**".format(df_norm['normal'].all())))
display(df_norm)
display(Markdown("Normal distribution **not** given for **all** distributions => *Wilcoxon test* instead of paired t-test"))

display(Markdown("**Wilcoxon tests**"))
display(wilc_phase.round(3))
display(Markdown("**For comparison: paired t-tests** (provides comparable results)"))
display(tt_phase.round(3))

##### Interpretation

"Even though the HR of the Intervention group was slightly decreased compared to the *Control* group during the *AT* and *FB* sub-phases both groups reached comparable stress levels especially during *MIST2* and *MIST3*. This finding is supported by significant HR increases during *AT* in each *MIST* phase for both groups (*Control*: Table 2, *Intervention*: Table 5). […] Thus, the MIST successfully caused sympathetic activation during *AT* and *FB* regardless of the condition."

### Question II: Does the intervention show an effect on the cardiovascular stress response?

#### Examination – I

It is examined whether the intervention causes significant differences between the two groups during the MIST during the *BL* subphase.

#### Statistics

##### Analysis

* Preparatory analysis:
    * Test for normal distribution (Shapiro-Wilk test)
    * Test for equality of variance (Levene test)
* * Test*: Mixed ANOVA
* *DV*: $\text{HR}_{rel}$ (normalized heart rate) during *BL* => since in this analysis no longer exclusively inner subject factors (repeated measurements) are considered, but rather group comparisons
* *Inner subject factor*: Phase
* *Inter subject factor*: Group
* *Measure of effect size*: $\eta_p^2$

=> Analogous analysis for the other HRV parameters

**Baseline**

In [ ]:
dv = "HR_rel"

df_analysis = df_hr.xs("BL", level='subphase')

# Test for normal distribution
df_norm = df_analysis.reset_index().groupby(by=["phase"]).apply(lambda df: pg.normality(data=df, dv=dv, group="condition"))
# Test for homoscedasticity
df_hom = df_analysis.reset_index().groupby(by=["phase"]).apply(lambda df: pg.homoscedasticity(data=df, dv=dv, group="condition"))

# Mixed ANOVA
anova_mixed = pg.mixed_anova(data=df_analysis.reset_index(), dv=dv, within="phase", between="condition", subject="subject")
# Post-hoc tests
posthoc_mixed = pg.pairwise_ttests(data=df_analysis.reset_index(), dv=dv, within="phase", between="condition", subject="subject", effsize="hedges", tail='one-sided', padjust='bonf')

# Output
display(Markdown("All normally distributed: **{}**".format(df_norm['normal'].all())))
display(df_norm.round(3))
display(Markdown("All variances (approximately) equal: **{}**".format(df_hom['equal_var'].all())))

display(Markdown("**Mixed ANOVA**"))
display(anova_mixed.round(3))

display(Markdown("**Post-hoc tests**"))
display(Markdown("Significant interaction effect => Considering group differences at each time point (phase * condition))"))
display(posthoc_mixed.round(3))

##### Interpretation

"Over the three *MIST* phases the *HR* during *BL* of the *Intervention* group remained roughly constant, whereas it showed strong increases over time in the *Control* group (Figure 5). This drift of baseline levels in the *Control* group is reflected by a significant interaction of *condition* and *MIST Phase* in HR and (Table 6). Post-hoc tests in Table 7 revealed that baselines started to significantly differ in *MIST2* and continued for *MIST3*."

#### Plots

In [ ]:
fig, ax = plt.subplots()
sns.boxplot(data=df_analysis.reset_index(), x="phase", y=dv, hue="condition", hue_order=["Control", "Intervention"], ax=ax)

sig_pairs = [
    (("MIST2", "Control"), ("MIST2", "Intervention")),
    (("MIST3", "Control"), ("MIST3", "Intervention")),
]

add_stat_annotation(
    ax=ax, data=df_analysis.reset_index(), 
    x="phase", y=dv, hue="condition", 
    box_pairs=sig_pairs,
    pvalue_thresholds=pvalue_thres,
    test="t-test_ind",
    hue_order=["Control", "Intervention"],
    comparisons_correction='bonferroni'
)

ax.set_title("Relative Heart Rate during BL")
ax.set_ylabel("$\Delta HR~[\%]$")
fig.tight_layout()

#### Examination – II

It is investigated whether the intervention causes significant differences between the two groups during the MIST during the *CFI* subphase.

#### Statistics

##### Analysis

* Preparatory analysis:
    * Test for normal distribution (Shapiro-Wilk test)
    * Test for equality of variance (Levene test)
* *Test*: Mixed ANOVA
* *DV*: $\text{HR}_{rel}$ (normalized heart rate) during *CFI* => since in this analysis not only inner subject factors (repeated measurements) are considered, but rather group comparisons
* *Inner-subject factor*: Phase
* *Inter-subject factor*: Group
* *Measure of effect size*: $\eta_p^2$

=> Analogous analysis for the other HRV parameters

In [ ]:
dv = "HR_rel"

df_analysis = df_hr.xs("CFI", level='subphase')

# Test for normal distribution
df_norm = df_analysis.reset_index().groupby(by=["phase"]).apply(lambda df: pg.normality(data=df, dv=dv, group="condition"))
# Test for homoscedasticity
df_hom = df_analysis.reset_index().groupby(by=["phase"]).apply(lambda df: pg.homoscedasticity(data=df, dv=dv, group="condition"))

# Mixed ANOVA
anova_mixed = pg.mixed_anova(data=df_analysis.reset_index(), dv=dv, within="phase", between="condition", subject="subject")

# Output
display(Markdown("Alle normalverteilt: **{}**".format(df_norm['normal'].all())))
display(df_norm.round(3))
display(Markdown("Alle Varianzen (in etwa) gleich: **{}**".format(df_hom['equal_var'].all())))

display(Markdown("**Mixed ANOVA**"))
display(anova_mixed.round(3))

##### Interpretation

"For the *CFI/RP* sub-phase ANOVA revealed no significant interaction effects, but significant main effects for *condition* (Table 6). These findings indicate that the Intervention group recovered better from acute stress than the Control group."

#### Plots

In [ ]:
dv = "HR_rel"
df_plot = df_hr.loc[pd.IndexSlice[:, :, :, ["BL", "CFI"]]]
fig, axs = plt.subplots(figsize=(10, 5), ncols=2)

for (key, df), ax in zip(df_plot.reset_index().groupby("subphase"), axs):
    sns.violinplot(data=df, x="phase", y=dv, hue="condition", hue_order=["Control", "Intervention"], split=True, ax=ax)
    ax.set_title("Relative Heart Rate during {}".format(key))
    ax.set_ylabel("$\Delta HR~[\%]$")

fig.tight_layout()

### Question III: Does the intervention show an effect on mood worsening?

#### Examination

It is examined whether the intervention creates significant differences between the two groups in terms of stress-induced worsening of the mood.

#### Statistics

##### Analysis
* Preparatory analysis:
    * Test for normal distribution (Shapiro-Wilk test)
    * Test for equality of variance (Levene test)
* * Test*: Mixed ANOVA
* *DV*: MDBF subscales
* *Inner-subject factor*: Time
* *Inter-subject factor*: Group
* *Measure of effect size*: $\eta_p^2$

In [ ]:
df_mood.head()

In [ ]:
dv = "MDBF"
# Selecting the data to be examined
df_analysis = df_mood


# Test for normal distribution
df_norm = df_analysis.reset_index().groupby(by=["type", "time"]).apply(lambda df: pg.normality(data=df, dv=dv, group="condition"))
# Test for homoscedasticity
df_homo = df_analysis.reset_index().groupby(by=["type", "time"]).apply(lambda df: pg.homoscedasticity(data=df, dv=dv, group="condition"))

# Mixed ANOVA per MDBF subscale
anova_mixed = df_analysis.reset_index().groupby(by=["type"]).apply(lambda df: pg.mixed_anova(data=df, dv=dv, within="time", between="condition", subject="subject"))

# Output
display(Markdown("All normally distributed: **{}**".format(df_norm['normal'].all())))
display(Markdown("All variances (approximately) equal: **{}**".format(df_hom['equal_var'].all())))

display(Markdown("**Mixed ANOVA**"))
display(anova_mixed.round(3))

##### Interpretation

"Subjects in the Intervention group experienced slightly decreased mood worsening compared to the Control group, especially in the *Awake-Tired* dimension (Control: $12.45\% ± 13.28\%$; Intervention: $7.84\% ± 20.38\%$). However, ANOVA revealed no significant interaction effects (*condition* by *time*) and no significant main effects for *conditon*."

### Question IV: Does the intervention show an effect on the endocrine stress response?

#### Examination – I

It is examined whether the intervention creates significant differences between the two groups in terms of endocrine stress response.

#### Statistics

##### Analysis
* Preparatory analysis:
    * Test for normal distribution (Shapiro-Wilk test)
* *Test*: Independent t-test (two-sided)
* *DVs*: 
    * $\text{max_inc}$ (maximum cortisol increase in percent)
    * $AUC_g$
    * $AUC_i$
    * $a_{S1S4}$
* *Measure of effect size*: Hedge's g

=> Analogous for other cortisol parameters ($\text{AUC}_g$, $\text{AUC}_i$, slope $a_{s1s4}$, ...)

In [ ]:
df_cort

In [ ]:
dv = "cortisol"
biomarker = ["max_inc_percent", "auc_g", "auc_i", "slope14"]
df_analysis = df_cort.loc[pd.IndexSlice[:, :, biomarker]]

# Test for normal distribution
df_norm = df_analysis.reset_index().groupby("biomarker").apply(lambda df: pg.normality(data=df, dv=dv, group="condition"))
df_norm_grp = pd.DataFrame(df_norm.reset_index().groupby("biomarker").apply(lambda df: df['normal'].all())).T

display(df_norm)
display(df_norm_grp)

# t-tests per biomarker
ttest_group = df_analysis.reset_index().groupby("biomarker").apply(lambda df: pg.pairwise_ttests(data=df, dv=dv, between="condition", subject="subject", effsize="hedges"))
# Mann-Whitney U tests per biomarker
mwu_group = df_analysis[dv].groupby(by="biomarker").apply(lambda df: pg.mwu(x=df.xs("Control", level="condition"), y=df.xs("Intervention", level="condition")))

# Output
display(Markdown("All normally distributed: **{}**".format(df_norm['normal'].all())))
display(Markdown("Normal distribution **not** given for **all** distributions => *Mann-Whitney-U-Test* instead of t-test for the non-normally distributed variables"))

for bm in biomarker:
    is_normal = df_norm_grp[bm].all()
    display(Markdown("* *{}* => Normal: **{}**".format(bm, is_normal)))
    if is_normal:
        display(ttest_group.loc[bm])
    else:
        display(mwu_group.loc[bm])

#display(Markdown("**Mann-Whitney-U-Tests**"))
#display(mwu_group.round(3))
#display(Markdown("**t-tests**"))
#display(ttest_group.round(3))

##### Interpretation

"Subjects in the Intervention group had significantly lower maximum cortisol responses compared to the Control group ($t(24) = 2.505, p = 0.019, d = 0.952$)."

#### Plots

In [ ]:
fig, axs = plt.subplots(ncols=len(biomarker), figsize=(12,5))
xlabel_map = {
    'auc_g': r"$AUC_G$",
    'auc_i': r"$AUC_I$",
    'max_inc_percent': r"$Inc_{max}$",
    'slope14': r"$a_{S1S4}$",
}
ylabel_map = {
    'auc_g': r"Cortisol AUC $\left[\frac{nmol \cdot min}{l}\right]$",
    'auc_i': r"Cortisol AUC $\left[\frac{nmol \cdot min}{l}\right]$",
    'max_inc_percent': r"Cortisol Change $[\%]$",
    'slope14': r"Cortisol Change $\left[\frac{nmol}{l \cdot min}\right]$",
}

for (key, df), ax in zip(df_analysis.reset_index().groupby("biomarker"), axs):
    sns.boxplot(data=df, x="condition", y="cortisol", ax=ax)
    ax.set_xlabel(xlabel_map[key])
    ax.set_ylabel(ylabel_map[key])
    

# stat annotation for significant features
for bm, i, test in zip(["max_inc_percent", "slope14"], [2, 3], ["t-test_ind", "Mann-Whitney"]):   
    add_stat_annotation(
        ax=axs[i], data=df_analysis.xs(bm, level="biomarker").reset_index(),
        x="condition", y=dv, 
        box_pairs=[("Control", "Intervention")],
        pvalue_thresholds=pvalue_thres,
        test=test,
        order=["Control", "Intervention"]
    )

fig.tight_layout()

#### Untersuchung – II
Es wird untersucht, ob die Intervention den Cortisolverlauf beeinflusst.

#### Statistik

##### Analyse
* Vorbereitende Analyse:
    * Test auf Normalverteilung (Shapiro-Wilk-Test)
    * Test auf Varianzgleichheit (Levene-Test)
* *Test*: Mixed ANOVA
* *AV*: Cortisol-Rohwerte
* *Maß für Effektstärke*: $\eta_p^2$

In [ ]:
dv = "cortisol"
# Entfernen von S0
df_analysis = df_cort_raw.drop('0', level='sample')

# Test auf Normalverteilung
df_norm = df_analysis.reset_index().groupby("sample").apply(lambda df: pg.normality(data=df, dv=dv, group="condition"))
# Test auf Homoskedastizität
df_hom = df_analysis.reset_index().groupby("sample").apply(lambda df: pg.homoscedasticity(data=df, dv=dv, group="condition"))

anova_mixed = pg.mixed_anova(data=df_analysis.reset_index(), dv=dv, between="condition", within="sample", subject="subject")

# Ausgaben
display(Markdown("Alle normalverteilt: **{}**".format(df_norm['normal'].all())))
display(df_norm.round(3))
display(Markdown("Alle Varianzen (in etwa) gleich: **{}**".format(df_hom['equal_var'].all())))

display(anova_mixed.round(3))

##### Interpretation

ANOVA further showed a significant interaction of *condition* by *time* for raw cortisol samples ($F(5,120) = 3.839, p = 0.003, \eta_p^2 = 0.138$).